In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split, ShuffleSplit
import pandas as pd
# from torch.utils.tensorboard import SummaryWriter
import time
import numpy as np

In [ ]:
#importamos o dataset (previamente baixado do Kaggle) e exibimos suas 5 primeiras linhas 
#com a função "head"
data = pd.read_csv('input/Skyserver_SQL2_27_2018 6_51_39 PM.csv')
data.head()

In [ ]:
#já a função unstitch separa os atributos da label, fazendo o oposto da função stitch
def unstitch(data):
    df = pd.DataFrame(data)
    return df[0].values.tolist(), df[1].values.tolist()

In [ ]:
#separamos o dataset original nos conjuntos de treino, validação e teste
#o conjunto de treino ficou com 60% dos exemplos, enquanto o de validação ficou com 20%,
#e o de testes também com 20%
train, test = train_test_split(data, test_size=0.2, shuffle=True)
train, val = train_test_split(train, test_size=0.25, shuffle=True)

In [ ]:
class Net(nn.Module):
    #em __init__ definimos cada uma das camadas
    def __init__(self, activate_dropout=False, weightDecay=0):
        super(Net, self).__init__()
        #fc1 corresponde à camada de entrada e terá 20 neurônios
        self.fc1 = nn.Linear(2, 20)
        #a função de ativação utilizada para cada neurônio é tanh
        self.rl1 = nn.Tanh()
        # sigmoid ou relu
        #fc2 corresponde à camada oculta, também tendo 20 neurônios
        self.fc2 = nn.Linear(20, 20)
        self.rl2 = nn.Tanh()
        #já fc3 corresponde à cama de saída, com apenas 3 neurônios, visto que lidamos com
        #3 classes na classificação
        self.fc3 = nn.Linear(20, 2)
        #a função de saída será softmax, recomendado para casos de classificação com múltiplas
        #classes
        # sigmoid
        self.smout = nn.Sigmoid()
        
        #a função de loss utilizada é L1 (MAE, ou erro absoluto médio)
        self.criterion = nn.L1Loss()
        #o otimizador selecionado foi Adam
        self.optimizer = optim.Adam(self.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        self.activate_dropout=activate_dropout
    def forward(self, x):
        #o PyTorch requer que o forward propagation seja declarado para cada camada
        x = self.fc1(x)
        if self.activate_dropout:
            x = self.do1(x)
        x = self.rl1(x)
        x = self.fc2(x)
        if self.activate_dropout:
            x = self.do2(x)
        x = self.rl2(x)
        x = self.fc3(x)
        if self.activate_dropout:
            x = self.do3(x)
        x = self.smout(x)
        return x
    def loss(self, loss_set):
        #a função de loss é definida
        inputs, labels = unstitch(loss_set)
        inputs = Variable(torch.FloatTensor(inputs))
        labels = Variable(torch.FloatTensor(labels))
        result = self(inputs)
        loss = self.criterion(result, torch.max(labels, 1)[1])

        return float(loss.item())
    def train(self, numEpochs, train):
        #função que realiza o treino da rede
        #o treino inicia com a separação dos parâmetros e das labels
        inputs, labels = unstitch(train)
        # usar pra separa a reposta(label) do input
        inputs = Variable(torch.FloatTensor(inputs))
        labels = Variable(torch.FloatTensor(labels))

        #em seguida, para cada época é realizado o forward propagation, e então o otimizador
        #efetua o back propagation atualizando os pesos de cada neurônio
        #os resultados de cada época são armazenados para exibição no TensorBoard
        for epoch in range(numEpochs):
            self.optimizer.zero_grad()
            outputs = self(inputs)
            self.loss = self.criterion(outputs, labels)
            self.loss.backward()    
            self.optimizer.step()

            self.store_results(epoch, train)
            self.print_final_train_acc(epoch, train)
    def accuracy(self, acc_set):
        #definição do cálculo da acurácia
        inputs, labels = unstitch(acc_set)
        result = self(Variable(torch.FloatTensor(inputs)))
        inputs_max = np.argmax(result.detach().cpu().numpy(), axis=1)
        labels_max = np.argmax(np.array(labels), axis=1)
        correct = np.sum(inputs_max == labels_max)

In [ ]:
REPORT_RATE = 10
STORE_RESULT = True
NUM_EPOCHS = 5000
BATCH_SIZE = 128
DROPOUT_RATE = 0.1
ACTIVATE_DROPOUT = False
WEIGHT_DECAY = 0
LEARNING_RATE = 0.01

#executamos o treinamento, verificando o tempo levado para que ele ocorra
t0=time.time()
# writer = SummaryWriter(log_dir='runs3/arch_3layer_final')
batchNet = Net(ACTIVATE_DROPOUT, WEIGHT_DECAY)
batchNet.train(NUM_EPOCHS, train)
t1=time.time()
# parametros utilizadsoq ue deram certo

#é criada uma linha para o arquivo CSV com os dados da execução atual
grid_search_df = pd.DataFrame({'layer_type': ["Linear"],
                               'layers': ["(6, 20), (20, 20), (10, 2)"],
                               'num_trainable_params':['630'],
                               'time': [str(round(t1-t0,3))],
                               'activation': ["TanH"],
                               'optimizer': ["Adam"],
                               'optimizer_params': ["actual weight_decay="+str(WEIGHT_DECAY)],
                               'learning_rate': ["actual learning_rate="+str(LEARNING_RATE)],
                               'loss_func': ["MSELoss"],
                               'activate_dropout': [str(ACTIVATE_DROPOUT)],
                               'dropout_rate': [str(DROPOUT_RATE)],
                               'num_epochs': [str(NUM_EPOCHS)],
                               'batch_size': [str(BATCH_SIZE)],
                               'train_accuracy': [str(round(batchNet.accuracy(train),4))],
                               'val_accuracy': [str(round(batchNet.accuracy(val),4))]
                               })

#a acurácia de validação é exibida
print(f"Validation accuracy: {batchNet.accuracy(val)}")
print("-"*10)

# a linha do arquivo CSV é efetivamente inserida no arquivo
file_df = pd.read_csv("grid_search.csv")
file_df = file_df.append(grid_search_df)
file_df.to_csv("grid_search.csv",index=False)
# writer.close()
# writer.flush()

In [ ]:
#após finalizar a busca pelos hiperparâmetros ideais, realizamos a checagem da acurácia
#para o conjunto de teste
print(f"Test accuracy: {batchNet.accuracy(test)}")